# Knative

Knative (ausgesprochen kay-nay-tiv) erweitert Kubernetes um eine Reihe von Middleware-Komponenten, die für die Erstellung moderner, container-basierter Anwendungen unerlässlich sind.

Knative Komponenten konzentrieren sich auf die Lösung alltäglicher, aber schwieriger Aufgaben wie:

* Bereitstellen eines Containers
* Routing und Verwaltung des Datenverkehrs mit blue / green Bereitstellung
* Automatische Skalierung des Workloads
* Bindung laufender Dienste an mehrere Ökosysteme

***
Sequence
--------

![](https://knative.dev/docs/eventing/samples/sequence/sequence-terminal/sequence-terminal.png)

Quelle: knative.dev
- - -

Basierend auf: https://github.com/knative/eventing-contrib/blob/master/cmd/appender/main.go

In [ ]:
! kubectl create namespace sequence
! # kubectl label namespace sequence-example knative-eventing-injection=enabled

- - - 
Erstellen der Knative Services

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: serving.knative.dev/v1
kind: Service
metadata:
  name: first
  namespace: sequence     
spec:
  template:
    spec:
      containers:
        - image: gcr.io/knative-releases/knative.dev/eventing-contrib/cmd/appender
          env:
            - name: MESSAGE
              value: " - Handled by 0"

---
apiVersion: serving.knative.dev/v1
kind: Service
metadata:
  name: second
  namespace: sequence     
spec:
  template:
    spec:
      containers:
        - image: gcr.io/knative-releases/knative.dev/eventing-contrib/cmd/appender
          env:
            - name: MESSAGE
              value: " - Handled by 1"
---
apiVersion: serving.knative.dev/v1
kind: Service
metadata:
  name: third
  namespace: sequence     
spec:
  template:
    spec:
      containers:
        - image: gcr.io/knative-releases/knative.dev/eventing-contrib/cmd/appender
          env:
            - name: MESSAGE
              value: " - Handled by 2"
%EOF%


Erstellen der Sequenzen

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: flows.knative.dev/v1beta1
kind: Sequence
metadata:
  name: sequence
  namespace: sequence     
spec:
  channelTemplate:
    apiVersion: messaging.knative.dev/v1beta1
    kind: InMemoryChannel
  steps:
    - ref:
        apiVersion: serving.knative.dev/v1
        kind: Service
        name: first
        namespace: sequence        
    - ref:
        apiVersion: serving.knative.dev/v1
        kind: Service
        name: second
        namespace: sequence        
    - ref:
        apiVersion: serving.knative.dev/v1
        kind: Service
        name: third
        namespace: sequence     
%EOF%


Als Event Auslöser verwenden wir `PingSource` 

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: sources.knative.dev/v1alpha2
kind: PingSource
metadata:
  name: ping-source
  namespace: sequence     
spec:
  schedule: "*/2 * * * *"
  jsonData: '{"message": "DatenSequence"}'
  sink:
    ref:
      apiVersion: flows.knative.dev/v1beta1
      kind: Sequence
      name: sequence
      namespace: sequence
%EOF%

In [ ]:
! kubectl get pods --namespace sequence

In [ ]:
%%bash
curl -v -H "Host: first.sequence.example.com" \
        -H "Content-Type: application/json" \
        istio-ingressgateway.istio-system -d '{"message": "DatenSequence"}'

In [ ]:
! kubectl -n sequence logs -l serving.knative.dev/service=first -c user-container 

Aufräumen

In [ ]:
! kubectl delete namespace sequence-example